# OTSU Big Data Cloud BE

Welcome to this training of OTSU Cloud, Big Data and Machine Learning module. In this

In [ ]:
import gcsfs
import rioxarray
import rasterio
import os
import numpy as np
from distributed import Client
import xarray as xr

In [ ]:
import os

def set_env():
    os.environ["GS_NO_SIGN_REQUEST"] = "YES"

set_env()

## Check you have access to GCS bucket

We can use gcsfs package, which mmimics a file system usage on an object storage.

In [ ]:
import gcsfs
fs = gcsfs.GCSFileSystem(bucket_name="supaero", token='anon')

In [ ]:
fs.ls('supaero/31TCH')

In [ ]:
fs.ls('supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2')

## How to read a product band using rasterio

Well read only a subsection of a product, using classical Numpy slices. This way, only selected pixels will be loaded into memory.

In [ ]:
xds_11 = rioxarray.open_rasterio("gs://supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2_FRE_B11.tif")
xds_11 = xds_11[:,4000:5000,4000:5000]
xds_11

In [ ]:
xds_11.plot()

## Building a single date Dataset

First, without using Dask, will build an Xarray Dataset from two DataArray that share the same dimensions and coordinates.

As we'll read two bands that to not share the same resolution, we'll have to resample the green band (using a simple mean).

Then, we'll use the Dataset to compute two more variable, NDSI, and snow mask.

In [ ]:
green = rioxarray.open_rasterio("gs://supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2_FRE_B3.tif")
green = green[:,8000:10000,8000:10000]
# Rééchantillonage à 20m, diviser résolution par 2
green = green.coarsen(x=2, y=2, boundary='pad').mean()
#No data
green = green.where(green != -10000)
green.rio.write_nodata(-10000, encoded=True, inplace=True)
green

In [ ]:
green.plot()

In [ ]:
swir = rioxarray.open_rasterio("gs://supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2_FRE_B11.tif")
swir = swir[:,4000:5000,4000:5000]
#No data
swir = swir.where(swir != -10000)
swir.rio.write_nodata(-10000, encoded=True, inplace=True)
swir

In [ ]:
swir.plot()

In [ ]:
ndsi = (green - swir) / (green + swir)
ndsi

In [ ]:
ndsi.plot()

In [ ]:
(ndsi > 0.4).plot()

In [ ]:
sub_ds = xr.Dataset({"green": green, "swir": swir})
sub_ds

In [ ]:
sub_ds["ndsi"] = (sub_ds.green - sub_ds.swir) / (sub_ds.green + sub_ds.swir)
sub_ds["snow"] = sub_ds.ndsi > 0.4
sub_ds

In [ ]:
sub_ds.snow.plot()

### Compute snow percentage over the area

In [ ]:
sub_ds.snow.sum() / sub_ds.snow.size

## Compute a time series analysis

Now that we know how to create a single timestamp dataset, we'll build a dataset with a time dimension.
The idea is to stack single temporal datasets into a single one using a new time dimension.

Up to now, we've only built datasets that easily fit in memory, by taking only part of one observation. 
In order to be able to work on full images and on ten products, we'll need to use Dask.

In [ ]:
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GiB')
client

In [ ]:
client.run(set_env)

In [ ]:
def read_one_band(product, band, coarsen=1):
    chunks=(-1, 1024*coarsen, 1024*coarsen)
    band = rioxarray.open_rasterio(f"gs://supaero/31TCH/{product}/{product}_FRE_{band}.tif", 
                                chunks=chunks,
                                lock=False).squeeze('band', drop=True)
    band = band.where(band != -10000)
    band.rio.write_nodata(-10000, encoded=True, inplace=True)
    if coarsen > 1:
        band = band.coarsen(x=coarsen, y=coarsen, boundary='pad').mean()
    return band

In [ ]:
read_one_band("SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2", "B3", 2)

In [ ]:
def create_dataset(product):
    ds = xr.Dataset({"green": read_one_band(product, "B3", 2), "swir": read_one_band(product, "B11")})
    ds["ndsi"] = (ds.green - ds.swir) / (ds.green + ds.swir)
    ds["snow"] = ds.ndsi > 0.4
    return ds

In [ ]:
create_dataset("SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2")

In [ ]:
product_list = [path.split("/")[-1] for path in fs.ls('supaero/31TCH')]

In [ ]:
datasets = []
for product in product_list:
    datasets.append(create_dataset(product))

In [ ]:
# Create time index
dates = [product.split("_")[1] for product in product_list]
dates

In [ ]:
import pandas as pd
dt_index = pd.to_datetime(dates, format="%Y%m%d-%H%M%S-%f")
dt_index.name = "time"
dt_index

In [ ]:
complete_ds = xr.concat(datasets, dt_index)
complete_ds

### Compute snow percentage time series over the whole image

In [ ]:
snow_percentage = (complete_ds.snow.sum(dim=["x", "y"]) / complete_ds.ndsi.count(dim=["x", "y"])).compute()

In [ ]:
snow_percentage

In [ ]:
snow_percentage.sortby("time").plot()

In [ ]:
snow_percentage.sortby("time").plot.step(where="mid")

# Extends other statistics

Snow area
NDSI
Add elevation, and compute snowline